# Simple Marlowe Contract for a Token Airdrop

**Executive Summary**

This contract demonstrates the use of Cardano address in a Marlowe contract, where formerly public-key hashes were used to identify non-role parties in a contract. The contract itself is a simple airdrop of tokens to three recipients, where the airdrop must occur after a specified time (via a `Notify` containing a `TimeIntervalStart` condition).

Here is a Blockly version of the contract (but with public key hashes appearing instead of addresses because Blockly does not yet display addresses).
![Token drop contract](token-drop.png)

## Preliminaries

### Record versions

In [1]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [2]:
marlowe-cli --version

marlowe-cli 0.0.8.1


In [3]:
git rev-parse HEAD

6c115514abcd567c24cfe70a847b6d3a916f110f


### Select the network

In [4]:
export CARDANO_TESTNET_MAGIC=2
export CARDANO_NODE_SOCKET_PATH=node.socket

### Set the folder where keys are stored.

In [5]:
export TREASURY=treasury

### Find the current time and compute time constants

In [6]:
NOW=$(($(date -u +%s) * 1000))
echo "$NOW"

1663467698000


### Constants

#### Time constants

In [7]:
SECOND=1000
MINUTE=$((60 * SECOND))
HOUR=$((60 * MINUTE))
DAY=$((24 * HOUR))

#### Lovelace to Ada

In [8]:
ADA=1000000

### Credentials for the token distribuor

In [9]:
PAYMENT_SKEY="$TREASURY/payment.skey"
PAYMENT_ADDR="$(cat $TREASURY/payment.testnet.address)"
echo "$PAYMENT_ADDR"

addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j


### Tokens to be distributed

These have already been minted . . .

In [10]:
POLICY_ID=8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
TOKEN_NAME=M4B

. . . and are in the distributor's wallet.

In [11]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$PAYMENT_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
034e51a5237b5cb35c71024439a3694ae7433aa573e92257a4001432279533c6     0        218623235553 lovelace + TxOutDatumNone
70af7d24ac3ad95dc0a4a8d8ae3027807993e4a2095fbf708a605104104de99f     0        3000000 lovelace + 3 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d3442 + TxOutDatumNone


### Recipients of the tokens

In [12]:
RECIPIENT_1=addr_test1qqjrjfslev50d5jn6qdeztxsqvuaaefq6e4hu55n59grjj7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqwhulyr
RECIPIENT_2=addr_test1qraxynufcduk7ak7nke2jm6pnc32vn7wsmm7ml30cutfgg7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq77wfgp
RECIPIENT_3=addr_test1qzenwj7elwatk3mmc368mwnv067fqnuk3x7u4nqw5dezg8wr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqf6k36p

## Setup

Create a contract that pays the tokens to the recipients after a specified time.

In [13]:
yaml2json << EOI > token-drop-1.contract
when:
- case:
    notify_if:
      ge_than: $((NOW + 5 * MINUTE))
      value: time_interval_start
  then:
    pay: 1
    token:
      currency_symbol: $POLICY_ID
      token_name: "$TOKEN_NAME"
    from_account:
      address: $PAYMENT_ADDR
    to:
      party:
        address: $RECIPIENT_1
    then:
      pay: 1
      token:
        currency_symbol: $POLICY_ID
        token_name: "$TOKEN_NAME"
      from_account:
        address: $PAYMENT_ADDR
      to:
        party:
          address: $RECIPIENT_2
      then:
        pay: 1
        token:
          currency_symbol: $POLICY_ID
          token_name: "$TOKEN_NAME"
        from_account:
          address: $PAYMENT_ADDR
        to:
          party:
            address: $RECIPIENT_3
        then: close
timeout: $((NOW + 5 * DAY))
timeout_continuation: close
EOI
cat token-drop-1.contract

{"timeout":1663899698000,"timeout_continuation":"close","when":[{"case":{"notify_if":{"ge_than":1663467998000,"value":"time_interval_start"}},"then":{"from_account":{"address":"addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j"},"pay":1,"then":{"from_account":{"address":"addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j"},"pay":1,"then":{"from_account":{"address":"addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j"},"pay":1,"then":"close","to":{"party":{"address":"addr_test1qzenwj7elwatk3mmc368mwnv067fqnuk3x7u4nqw5dezg8wr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqf6k36p"}},"token":{"currency_symbol":"8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d","token_name":"M4B"}},"to":{"party":{"address":"addr_test1qraxynufcduk7ak7nke2jm6pnc32vn7wsmm7ml30cutfgg7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq77wfgp"}},"token":{"currency_symbol":"8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d","token_name":"M4B"}},"to":{"party":{"address":"

We create the initial state with the tokens already at the contract's address.

In [14]:
yaml2json << EOI > token-drop-1.state
accounts:
- - - address: $PAYMENT_ADDR
    - currency_symbol: ''
      token_name: ''
  - $((3 * ADA))
- - - address: $PAYMENT_ADDR
    - currency_symbol: $POLICY_ID
      token_name: "$TOKEN_NAME"
  - 3
boundValues: []
choices: []
minTime: 0
EOI
cat token-drop-1.state

{"accounts":[[[{"address":"addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j"},{"currency_symbol":"","token_name":""}],3000000],[[{"address":"addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j"},{"currency_symbol":"8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d","token_name":"M4B"}],3]],"boundValues":[],"choices":[],"minTime":0}


## Transaction 1: Create the contract by depositing the tokens

Create the `.marlowe` file that contains the initial contract and state information.

In [15]:
marlowe-cli run initialize --contract-file token-drop-1.contract \
                           --state-file    token-drop-1.state    \
                           --out-file      token-drop-1.marlowe

Submit the transaction to make the deposit.

In [16]:
TX_1=$(
marlowe-cli run auto-execute --marlowe-out-file token-drop-1.marlowe \
                             --change-address "$PAYMENT_ADDR"        \
                             --required-signer "$PAYMENT_SKEY"       \
                             --out-file token-drop-1.txbody          \
                             --submit 60                             \
                             --print-stats                           \
| sed -e 's/^TxId "\(.*\)"$/\1/'                                     \
)
echo 'TxId "'$TX_1'"'


Fee: Lovelace 224813
Size: 1131 / 16384 = 6%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%
TxId "9d5bbf45591baca23e029dcf9214d994dde1a288e51d9ac027d68d80bed2d66b"


Now compute the script address and look to see that the tokens have been deposited there.

In [17]:
SCRIPT_ADDR=$(jq -r '.tx.marloweValidator.address' token-drop-1.marlowe)
echo "$SCRIPT_ADDR"

addr_test1wrv0vwr4megau50ujjwsktvajmsu6dzza2rnalufd3husaqs9v6rv


In [18]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$SCRIPT_ADDR" | sed -n -e "1,2p;/$TX_1/p;"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
9d5bbf45591baca23e029dcf9214d994dde1a288e51d9ac027d68d80bed2d66b     1        3000000 lovelace + 3 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d3442 + TxOutDatumHash ScriptDataInBabbageEra "a24ab48ea5e1c8c75d33c6583fbce3ed29a2ad837afe5d7a0501e19bc685ad47"


See that the tokens are gone from the distributor's wallet.

In [19]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$PAYMENT_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
9d5bbf45591baca23e029dcf9214d994dde1a288e51d9ac027d68d80bed2d66b     0        218623010740 lovelace + TxOutDatumNone


The recipients have not been party to this transaction.

In [20]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$RECIPIENT_1" --address "$RECIPIENT_2" --address "$RECIPIENT_3" | sed -n -e "1,2p;/$TX_1/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


## Transaction 2: Airdrop the tokens to the recipients

Wait five minutes for the time when the airdrop is allowed.

In [21]:
sleep 5m

We just need to notify the contract 

In [22]:
marlowe-cli run prepare --marlowe-file token-drop-1.marlowe                      \
                        --notify                                                 \
                        --invalid-before    "$((NOW + 5 * MINUTE + 1 * SECOND))" \
                        --invalid-hereafter "$((NOW + 1 * HOUR))"                \
                        --out-file token-drop-2.marlowe

Payment 1
  Acccount: Address ""addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j""
  Payee: Party (Address ""addr_test1qqjrjfslev50d5jn6qdeztxsqvuaaefq6e4hu55n59grjj7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqwhulyr"")
  Ada: Lovelace {getLovelace = 0}
  8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d."M4B": 1
Payment 2
  Acccount: Address ""addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j""
  Payee: Party (Address ""addr_test1qraxynufcduk7ak7nke2jm6pnc32vn7wsmm7ml30cutfgg7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq77wfgp"")
  Ada: Lovelace {getLovelace = 0}
  8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d."M4B": 1
Payment 3
  Acccount: Address ""addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j""
  Payee: Party (Address ""addr_test1qzenwj7elwatk3mmc368mwnv067fqnuk3x7u4nqw5dezg8wr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqf6k36p"")
  Ada: Lovelace {getLovelace = 0}
  8bb3b343d8e404472337966a722150048c768d0a92a9813596c5

Submit the transaction to make the airdrop.

In [23]:
TX_2=$(
marlowe-cli run auto-execute --tx-in-marlowe "$TX_1#1"               \
                             --marlowe-in-file  token-drop-1.marlowe \
                             --marlowe-out-file token-drop-2.marlowe \
                             --change-address "$PAYMENT_ADDR"        \
                             --required-signer "$PAYMENT_SKEY"       \
                             --out-file token-drop-2.txbody          \
                             --submit 60                             \
                             --print-stats                           \
| sed -e 's/^TxId "\(.*\)"$/\1/'                                     \
)
echo 'TxId "'$TX_2'"'


Fee: Lovelace 1586594
Size: 14079 / 16384 = 85%
Execution units:
  Memory: 10431296 / 14000000 = 74%
  Steps: 2698172655 / 10000000000 = 26%
TxId "5f16523ce9acb2fae2da0caabc649b2c7d832cf3dbd5ba4179252d2be8584c47"


The script address is empty.

In [24]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$SCRIPT_ADDR" | sed -n -e "1,2p;/$TX_1/p;/$TX_2/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


The distributor received some change, but no tokens.

In [25]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$PAYMENT_ADDR" | sed -n -e "1,2p;/$TX_1/p;/$TX_2/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
5f16523ce9acb2fae2da0caabc649b2c7d832cf3dbd5ba4179252d2be8584c47     0        218617997696 lovelace + TxOutDatumNone
5f16523ce9acb2fae2da0caabc649b2c7d832cf3dbd5ba4179252d2be8584c47     1        3000000 lovelace + TxOutDatumNone


The first recipient recieved their token.

In [26]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$RECIPIENT_1" | sed -n -e "1,2p;/$TX_1/p;/$TX_2/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
5f16523ce9acb2fae2da0caabc649b2c7d832cf3dbd5ba4179252d2be8584c47     2        1142150 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d3442 + TxOutDatumNone


The second recipient received their token.

In [27]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$RECIPIENT_2" | sed -n -e "1,2p;/$TX_1/p;/$TX_2/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
5f16523ce9acb2fae2da0caabc649b2c7d832cf3dbd5ba4179252d2be8584c47     4        1142150 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d3442 + TxOutDatumNone


The third recipient recieved their token.

In [28]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$RECIPIENT_3" | sed -n -e "1,2p;/$TX_1/p;/$TX_2/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
5f16523ce9acb2fae2da0caabc649b2c7d832cf3dbd5ba4179252d2be8584c47     3        1142150 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d3442 + TxOutDatumNone
